In [ ]:
# 1. csv파일을 load

# 2. csv파일의 '영엄장명' column을 읽어서 네이버 지도검색 url에서 넣어서 검색

# 3. 메뉴테그를 순서대로 읽은 뒤 대표메뉴 3가지와 가격을 추출 후 df에 저장
# column: 'menu_1_name, menu_1_price, menu_2_name, menu_2_price, menu_3_name, menu_3_price'

# 4 df를 csv파일로 저장